# String similarity by Levenshtein Distance
This script aims to compute the similarity between strings via Levenshtein Distance (https://en.wikipedia.org/wiki/Levenshtein_distance). It compares the names of streets and places from a current dataset with the ones obtained from the preceding OCR of labels within a historical map.

In [1]:
# load libraries
import pandas as pd
import geopandas as gpd
import itertools
import Levenshtein
import difflib
import fuzzywuzzy
from fuzzywuzzy import fuzz # compares TWO strings
from fuzzywuzzy import process # compares a string to MULTIPLE other strings

## import road network dataset of the city of Hamburg
Hamburg's road network ('Straßen- und Weggenetz Hamburg') provided via the Transparenzportal Hamburg (source: https://suche.transparenz.hamburg.de/dataset/strassen-und-wegenetz-hamburg-hh-sib9, last update in February 2018) can be obtained via WFS (http://archiv.transparenz.hamburg.de/hmbtgarchive/HMDK/hh_wfs_strassen_und_wegenetz_24055_snap_3.XML). The included layer 'strassennetz-gesamt' is used for further work.

Before going on working with the dataset, we dissolve it by its street names (column 'strassenname') in QGIS in order to have a single entry for each road:<br><br>
**QGIS Dissolve**
- input: strassennetz-gesamt
- dissolve field: strassenname
- dissolved: Strassen_Wegenetz_HH_dissolvedByName.shp

Also with the help of QGIS, a new field 'centroid_s' is created in Strassen_Wegenetz_HH_dissolvedByName.shp where the WKT coordinates of the approximate "centroid" of each resulting (Multi)LineString is stored.
We compute those centroids as follows:
- If a road within the original dataset (strassennetz-gesamt) consists of only one linestring, the point at half-length of this linestring is stored as 'centroid_s' in Strassen_Wegenetz_HH_dissolvedByName.shp
- If a road within the original dataset (strassennetz-gesamt) consists of exactly two linestrings, the interpolated point at half-length over both linestrings is stored as 'centroid_s' in Strassen_Wegenetz_HH_dissolvedByName.shp
- If a road within the original dataset (strassennetz-gesamt) consists of three or more linestrings, the centroid of their common rectangular bounding box is is stored as 'centroid_s' in Strassen_Wegenetz_HH_dissolvedByName.shp.

The number of linestrings per street was calculated within the original dataset (strassennetz-gesamt) by counting the number of identical street names (column 'strassenname'):

**QGIS Statistics by category**<br>
- input vector layer: strassennetz-gesamt
- field to calculate staistics on: strassenname
- fields with categories: strassenname

The field 'count' within the output table lists the number of linestrings per street name.

In [2]:
# import Straßen- und Wegenetz Hamburg - dissolved by street names
strassennetz = gpd.read_file("data/Strassen_Wegenetz_HH_dissolvedByName.shp")
strassennetz.head()

,von_netzkn,nach_netzk,von_statio,bis_statio,von_km,bis_km,klasse,strassensc,wegenummer,strassenna,...,ast,europastra,fahrstreif,fahrstre_1,fahrstre_2,bahnigkeit,geschwindi,wegeart,centroid_s,geometry
0,242613570,242613547,0,290,None,None,G,S008,G 5870,Sachsenstraße,...,0,None,1,0,1,1,50 km/h,"Stadtstraße (alle Straßen, die nicht BAB oder ...",Point (10.0264443 53.54629129),"LINESTRING (10.02463 53.54566, 10.02484 53.545..."
1,242515476,242515485,0,112,None,None,G,M175,G 4620,Mewesweg,...,0,None,1,0,0,1,30 km/h,"Stadtstraße (alle Straßen, die nicht BAB oder ...",Point (9.87681741 53.53467149),"LINESTRING (9.87609 53.53441, 9.87743 53.53489..."
2,232510181,232511346,0,94,None,None,G,W526,G 258,Walter-Jungleib-Straße,...,0,None,1,0,1,1,30 km/h,"Stadtstraße (alle Straßen, die nicht BAB oder ...",Point (9.90921235 53.64648665),"LINESTRING (9.90934 53.64606, 9.90912 53.64682..."
3,242617259,242617260,0,134,None,None,G,M322,G 4759,Mühlenstieg,...,0,None,0,0,0,0,None,unbekannt,Point (10.06146359 53.573119),"MULTILINESTRING ((10.05969 53.57264, 10.06349 ..."
4,232510437,232510438,0,20,None,None,G,B422,G 938,Boberstraße,...,0,None,1,0,1,1,30 km/h,"Stadtstraße (alle Straßen, die nicht BAB oder ...",Point (9.88700469 53.60026245),"LINESTRING (9.88783 53.60103, 9.88780 53.60095..."


In [3]:
# # get all column names
# for col in strassennetz.columns: 
#     print(col)

## clean up the road network dataset
As we only need the geometry, centroids, and names of the streets we can drop all remaining columns in *strassennetz*.
As shapefiles have an attribute name limit of 10 characters, we rename the affected columns to improve the readability.

In [4]:
# extract only the few needed columns
strassennetz = strassennetz[['strassenna','geometry','centroid_s']]
# rename columns
strassennetz=strassennetz.rename(columns={"strassenna":"strassenname",
                                         "centroid_s":"geometry_centroid"})
strassennetz

,strassenname,geometry,geometry_centroid
0,Sachsenstraße,"LINESTRING (10.02463 53.54566, 10.02484 53.545...",Point (10.0264443 53.54629129)
1,Mewesweg,"LINESTRING (9.87609 53.53441, 9.87743 53.53489...",Point (9.87681741 53.53467149)
2,Walter-Jungleib-Straße,"LINESTRING (9.90934 53.64606, 9.90912 53.64682...",Point (9.90921235 53.64648665)
3,Mühlenstieg,"MULTILINESTRING ((10.05969 53.57264, 10.06349 ...",Point (10.06146359 53.573119)
4,Boberstraße,"LINESTRING (9.88783 53.60103, 9.88780 53.60095...",Point (9.88700469 53.60026245)
...,...,...,...
9222,Unbenannter Weg Hochwasserschutzanlage St. Pau...,"MULTILINESTRING ((9.95378 53.54550, 9.95386 53...",Point (9.95698872 53.54581077)
9223,Große Reichenstraße,"LINESTRING (9.99370 53.54854, 9.99376 53.54855...",Point (9.99527703 53.54847137)
9224,Erna-Stahl-Ring,"MULTILINESTRING ((10.04644 53.62976, 10.04533 ...",Point (10.04810995 53.62980422)
9225,Waltershofer Straße,"MULTILINESTRING ((9.89889 53.47866, 9.89897 53...",Point (9.90611363 53.49121981)


## set up the string similarity

In [5]:
# String similarity  T E S T:
# search for an EXACT buzzword ('Katharinen') in strassenname
strassennetz[strassennetz['strassenname'].str.contains('Katharinen',case=False)] # case insensitive

,strassenname,geometry,geometry_centroid
3288,Katharinenkirchhof,"LINESTRING (9.99389 53.54527, 9.99415 53.54538...",Point (9.99467475 53.54556798)
3601,Katharinenfleet,"MULTILINESTRING ((9.98949 53.54527, 9.98961 53...",Point (9.99141429 53.54525161)
3697,Katharinenbrücke,"LINESTRING (9.99297 53.54581, 9.99309 53.54590...",Point (9.99313854 53.54594145)
5684,Katharinenstraße,"LINESTRING (9.98926 53.54602, 9.98935 53.54604...",Point (9.99111724 53.54596565)


### store all street names in a list
and, at the same time, drop all entries which have an unknown street name. Their 'strassenname' is represented by the prefix "Unbenannte".<br>
The list format enables an improved comparison for the levenshtein distance which is able to compare a single string (here: a historical street name) to a list of strings (here: *strassen_list*).

In [6]:
strassen_list = strassennetz['strassenname'].tolist()

# delete all roads containting "Unbenannte"
# if there is this prefix within a road name
substring = "Unbenannte"

for strasse in sorted(strassen_list):
    if substring in strasse:
        strassen_list.remove(strasse)
        
# sort *strassen_list* ascending        
for strasse in sorted(strassen_list):
    print(strasse)

1. Hafenstraße
2. Hafenstraße
ABC-Straße
Aalheitengraben
Aalkrautweg
Aalort
Aalwisch
Aalwischkoppel
Abbestraße
Abelke-Bleken-Ring
Abelskamp
Abendrothsweg
Abrahamstraße
Abteistraße
Achter Billing
Achter Lüttmoor
Achter de Höf
Achter de Kark
Achter de Wisch
Achterdwars
Achterkamp
Achtern Barls
Achtern Born
Achtern Brack
Achtern Hoff
Achtern Hollerbusch
Achtern Moor
Achtern Sand
Achtern Styg
Achterschlag
Ackermannstraße
Ackerstieg
Ackerweg
Adalbert-Stifter-Weg
Adalbertstraße
Adebarweg
Adenauerallee
Adickesstraße
Adlerhorst
Adlerstraße
Admiralitätstraße
Adolf-Köster-Damm
Adolf-Wagner-Straße
Adolf-von-Elm-Hof
Adolph-Schomacker-Weg
Adolph-Schönfelder-Straße
Adolphsbrücke
Adolphsplatz
Afrikahöft
Afrikastraße
Agathe-Lasch-Weg
Agathenstraße
Agnes-Gierck-Weg
Agnes-Wolffson-Straße
Agnesstraße
Ahlbecker Weg
Ahlfeld
Ahornallee
Ahornkamp
Ahornstraße
Ahornweg
Ahrensburger Platz
Ahrensburger Stieg
Ahrensburger Straße
Ahrensburger Weg
Ahrensfelder Weg
Ahrenshooper Straße
Akazienallee
Akazienweg
Akeleiw

Beim Schillingstift
Beim Schlump
Beim Schröderschen Hof
Beim Schäferhof
Beim Strohhause
Beim Trichter
Beim Wilhelmsburger Wasserturm
Beim Ziegelhof
Beimoorstraße
Beisserstraße
Bekassinenau
Bekkamp
Bekkampsweg
Bekkoppeln
Bekstück
Bekweg
Bekwisch
Belemannweg
Belgarder Straße
Bellealliancestraße
Bellerbek
Bellevue
Bellmannstraße
Beltgens Garten
Benatzkyweg
Bendestorfer Ring
Bendixensweg
Bengelsdorfstieg
Bengelsdorfstraße
Benittstraße
Bennigsenstraße
Benselweg
Benzenbergweg
Benzstraße
Berberweg
Berchtungweg
Berenberg-Gossler-Weg
Bergdoltweg
Bergedorfer Heerweg
Bergedorfer Markt
Bergedorfer Schloßstraße
Bergedorfer Straße
Bergheide
Bergiusstraße
Bergkoppelweg
Bergmannring
Bergmannstraße
Bergstedter Alte Landstraße
Bergstedter Chaussee
Bergstedter Kirchenstraße
Bergstedter Markt
Bergstieg
Bergstraße
Bergstraße (Straße im Bereich Friedhof Ohlsdorf keine amtl. Benennung!)
Bergwetternweg
Bergwinkel
Berkefeldweg
Berlepschweg
Berliner Platz
Berliner Tor
Berliner Ufer
Berlinertordamm
Bernadottestr

Flaßbarg
Flaßheide
Flaßmoor
Flaßweg
Flebbestraße
Fleestedter Straße
Fleetdamm
Fleetplatz
Flemingstraße
Flensburger Straße
Flerrenkamp
Flerrentwiete
Flethmannskamp
Flett
Fliederweg
Floot
Flora-Neumann-Straße
Florapark
Flotowstraße
Flottbeker Drift
Flottbeker Marktweg
Flottbeker Stieg
Flottbektal
Flughafenstraße
Flurkamp
Flurstraße
Flutende
Flüggenhofstieg
Flüggestraße
Flünkentwiete
Fockenweide
Focksweg
Fohlenweide
Fohrwisch
Fontanestraße
Fontenay
Fontenay-Allee
Foorthkamp
Foortstegel
Forbacher Straße
Forsmannstraße
Forsteck
Forstgrund
Forsthöhe
Forstweg
Foßberger Moor
Foßholt
Foßredder
Foßsölen
Fraenkelstraße
Frahmredder
Frahmstraße
Framheinstraße
Francoper Hinterdeich
Francoper Straße
Frankenstraße
Frankring
Franz-Gartmann-Treppe
Franz-Marc-Straße
Franz-Rohr-Weg
Franz-Röttel-Park
Franzosenheide
Franzosenkoppel
Frapanweg
Frascatiplatz
Frauenthal
Freegen
Freegenweg
Freesenstraße
Freesienweg
Freihafenelbbrücke
Freihafenstraße
Freiligrathstraße
Freiweide
Frettchenweg
Freudenthalweg
Frickes

Huulkamp
Huusbarg
Huusbargstieg
Huuskoppel
Huuskoppelstieg
Häherweg
Händelstraße
Hänselstieg
Häußlerstraße
Höfenkamp
Höfnageleck
Högenbarg
Högenkamp
Högenredder
Högenstraße
Högerdamm
Höhenstieg
Höhentwiete
Höhnerkamp
Höhnkoppel
Höhnkoppelort
Hölderlinsallee
Hölderlinstraße
Hölertwiete
Hölscherweg
Höltentwiete
Höltigbaum
Höltystraße
Hönermoor
Hönerstücken
Höpen
Höpengrund
Höpenstraße
Höperfeld
Höperstieg
Hörgensweg
Hörnumstraße
Hörstener Straße
Hörstenstieg
Hövelbrook
Hövelpromenade
Hövelweg
Höxterstraße
Hübbesweg
Hübenerkai
Hübenerstraße
Hügelhain
Hühnerposten
Hüllbeen
Hüllenkamp
Hüllenkoppel
Hülsdornweg
Hülsenstieg
Hültkoppel
Hültkoppelstieg
Hünefeldstraße
Hürthweg
Hüsermoor
Hütten
Ibsenweg
Ida-Boy-Ed-Straße
Ida-Ehre-Allee ehemals Krieger Ehrenallee (Straße im Bereich Friedhof Ohlsdorf keine amtl. Benennung!)
Ida-Ehre-Platz
Ifflandstraße
Igelgrund
Igelweg
Ihlestraße
Iland
Ilandkoppel
Ilenbrook
Ilenbuller
Ilenkamp
Ilenkruut
Ilenwisch
Ilextwiete
Ilexweg
Ilkstraat
Illerweg
Illiesweg
Iloh

Mensingstraße
Menzelstraße
Mercedesstraße
Merckelweg
Mergelgrund
Mergellstraße
Mergenthalerweg
Meriandamm
Meridianstraße
Merkatorweg
Merkenstraße
Merkurring
Merlingasse
Merowingerweg
Messeplatz
Messetunnel Sternschanze
Mesterbrooksweg
Mesterfeld
Mesterfeldweg
Mesterkamp
Mestorfweg
Methfesselstraße
Mette-Harden-Straße
Mettlerkampsweg
Metzenberg
Metzendorfer Weg
Metzer Straße
Meurerweg
Meuronstieg
Mewesweg
Mexikoring
Meyer-Delius-Platz
Meyerbeerstraße
Meyerhofstraße
Meyermannweg
Meyerstraße
Meßberg
Michael-Hering-Weg
Michael-Pritzl-Weg
Michaelisbrücke
Michaelispassage
Michaelisstraße
Michel-Nathan-Weg
Michelsenweg
Middeltwiete
Middendorfstraße
Milcherstraße
Milchgrund
Milchstraße
Mildestieg
Millerntordamm
Millerntorplatz
Millöckerweg
Mimeweg
Mindelweg
Mindermannweg
Minenstraße
Minnerstieg
Minnerweg
Minsbekkehre
Minsbekweg
Mirabellenweg
Mirowstraße
Mispelstieg
Mispelweg
Missundestraße
Mistralstraße
Mittelallee (Straße im Bereich Friedhof Ohlsdorf keine amtl. Benennung!)
Mittelhövel
Mittel

Steintwiete
Steintwietenhof
Steinwegel
Steinwegenskoppel
Steinwegpassage
Steinwerder Damm
Steinwerder Kai
Steinwiesenweg
Stellaustieg
Stellbrinkweg
Stellinger Chaussee
Stellinger Markt
Stellinger Steindamm
Stellinger Weg
Stellmacherstraße
Stellmannkamp
Stemmeshay
Stemwarder Straße
Stengelestraße
Stengeletwiete
Stenvort
Stenzelring
Stephansplatz
Stephanstraße
Stephanusgarten
Sternbergweg
Sterndoldenweg
Sternmoosweg
Sternschanze
Sternstraße
Sterntalerstraße
Sterntwiete
Stettiner Ufer
Steubenstraße
Sthamerstraße
Stiefmütterchenweg
Stiegkamp
Stieglitzstraße
Stiegstück
Stiftstraße
Stiller Weg
Stillhorner Damm
Stillhorner Hauptdeich
Stillhorner Stegel
Stillhorner Weg
Stindeweg
Stintfang
Stockflethweg
Stockhausenstraße
Stockkamp
Stockmeyerstraße
Stockrosenweg
Stoeckhardtstraße
Stofferkamp
Stolbergstraße
Stolper Straße
Stolpmünder Straße
Stoltenbrücke
Stoltenpark
Stoltenstraße
Stolzweg
Stoppelfeld
Storchenheimweg
Storchenschnabelstieg
Storchenstieg
Storchenwiese
Stormarner Straße
Stormarnhöhe


### tests with different libraries computing the Levenshtein Distance

#### 1) Levenshtein library

In [7]:
# T E S T  Levenshtein library :
# ratio
Levenshtein.ratio('Auto','Apfel')

0.2222222222222222

The ratio outputs the similarity between the two input strings [0..1] (https://rawgit.com/ztane/python-Levenshtein/master/docs/Levenshtein.html#Levenshtein-ratio).

#### 2) difflib library

In [8]:
# T E S T  difflib library
# with an OCR output

# difflib's function *get_close_matches* gives the best matches among the possibilities in a list,
# sorted by similarity score, most similar first.

difflib.get_close_matches('„ame', strassen_list)

[]

difflib was tested with different results from OCR - the recognized labels from the subset of the historical map (https://maps.princeton.edu/catalog/harvard-g6299-h3-1853-l5) used within the paper. Unfortunately, the library did not produce satisfying results for many recognized labels.<br><br>
Therefore, as well as due to the fact that the Levenshtein library does not support lists as input data, we decided to make use of the **fuzzywuzzy library** which breaks through those limitations.

## Levenshtein Distance
We use the ***fuzzywuzzy*** library to get close matches within the current *strassen_list* (https://github.com/seatgeek/fuzzywuzzy). <br> <br>
Args in *extract*: <br>
&nbsp;    **query**: A string to match against <br>
&nbsp;    **choices**: A list or dictionary of choices, suitable for use with
             extract(). <br>
&nbsp;    **processor**: Optional function for transforming choices before matching.
               See extract(). <br>
&nbsp;    **scorer**: Scoring function for extract(). <br>
&nbsp;    **score_cutoff**: Optional argument for score threshold. No matches with
                  a score less than this number will be returned. Defaults to 0. <br>
&nbsp;    **limit**: Optional maximum for the number of elements returned. Defaults
        to 5.

After the text recongnition with Tesseract OCR, all OCR results from the used historical map were put into an Excel file. We import the column containing the outputted strings ("cropped") here.

In [9]:
ocr_results = pd.read_excel('data/OCR_results.xlsx', header=1)
ocr_output_list = ocr_results["cropped"].tolist()
ocr_output_list

['_Hollandıschey',
 'pferdemarkg',
 'Per e Rrgas)',
 '_ hohe\\',
 'klopfen markt',
 'Adolphs Br.',
 'Schmigdedss',
 'ATHHAUS',
 '[Rath',
 '‚Schul',
 '| MARKT',
 'DO ke A aaa',
 'I Beichei',
 'HTollandısche',
 'Catharıineıl',
 'Nicola;',
 'Schauenburee',
 'St Jacob!',
 'chopenstehl',
 'BurstaH',
 "Rathhau's",
 '>Speersort|',
 'IH ı,xt 63}',
 'Hoplensar',
 'OrOS|',
 'Alter!',
 'Gymi',
 'SEOSSEeT',
 'Wandrahm',
 '.t Petry!',
 'Breite',
 'Reichen',
 'Ness|',
 "S'Cathal",
 'Marki',
 'kener Mandr-',
 'ud',
 "nas'",
 'tacdıhaus',
 'Brook',
 'Str‘,',
 'Fisch',
 'Ce”)',
 'Km',
 'ho)']

To check one OCR output for its similarity with the current street names in *strassen_list*, we choose one string out of the *ocr_output_list*

In [10]:
ocr_output = ocr_output_list[4]
ocr_output

'klopfen markt'

### 1) partial ratio
*fuzz.partial_ratio* computes the similarity of a possibly shorter string within parts of the longer string

In [11]:
partial_ratio = process.extract(ocr_output, strassen_list, scorer=fuzz.partial_ratio, limit=30)
print(partial_ratio)

[('Hopfenmarkt', 91), ('Marktweg', 77), ('Hopfensack', 70), ('Eppendorfer Marktplatz', 69), ('Alsterdorfer Markt', 69), ('Bergedorfer Markt', 69), ('Niendorfer Marktplatz', 69), ('Parkplatz Niendorfer Marktplatz bei Haus 24', 69), ('Zum Markt', 67), ('Koppel', 67), ('Klotzenmoor', 64), ('Poppenbüttler Markt', 62), ('Wandsbeker Marktplatz', 62), ('Hummelsbütteler Markt', 62), ('Eimsbütteler Marktplatz', 62), ('Ottenser Marktplatz', 62), ('Nienstedtener Marktplatz', 62), ('Barmbeker Markt', 62), ('Neugrabener Markt', 62), ('Veddeler Marktplatz', 62), ('Kleine Marienstraße', 62), ('Neuer Pferdemarkt', 62), ('Bahrenfelder Marktplatz', 62), ('Wellingsbütteler Markt', 62), ('Wandsbeker Marktstraße', 62), ('Flottbeker Marktweg', 62), ('Rothenburgsorter Marktplatz', 62), ('Stellinger Markt', 62), ('Klotzenmoorstieg', 62), ('Steinbeker Marktstraße', 62)]


In [12]:
print(len(process.extract(ocr_output, strassen_list, scorer=fuzz.partial_ratio, limit=100000)))

8667


As fuzzywuzzy outputs too many (in this example here: 8667) possibly similar strings, we have to define further "recommendations" or conditions to narrow and improve the results from the applied levenshtein distance algorithm.<br><br>
Also, for the subsequent approximate georeferencing - mentioned in section 3.6 within the paper - we want to extract those recognized historical labels having the best similarity to current street names - allover the historical map.

#### Give recommendation 1: only use matches with >= 75% out of *partial_ratio*
The threshold at 75% is manually defined and based on empirical values from tests with different labels.

In [13]:
for pr in partial_ratio:
    #print(pr[1])
    if all(pr[1] < 75 for pr in partial_ratio):
        print('\033[91m', # red
              '\033[1m', # bold
              "Don't use this label for the initial georeferencing, there is no match >= 75%!",
              2*"\n",
              " SKIP IT!")
        break
    elif any(pr[1] >= 75 for pr in partial_ratio):
        print("There is at least one match with >= 75% for this label :)",
              2*"\n",
              "You can go on with this label and check for further conditions"
             )
        break

There is at least one match with >= 75% for this label :) 

 You can go on with this label and check for further conditions


Next, print all matches with a similarity >= 75%

In [14]:
partial_ratio_above75 = []

for pr in partial_ratio:
    #print(pr[1])
    if (pr[1] >= 75):
        partial_ratio_above75.append(pr)
    elif all(pr[1] < 75 for pr in partial_ratio):
        print('\033[91m', # red
              '\033[1m', # bold
              "As i said: There is no match >= 75%! :(", 2*"\n",
              " SKIP THIS LABEL!")
        break

if partial_ratio_above75: # if the list *partial_ratio_above75* is not empty...
    if len(partial_ratio_above75) == 1:
        print('\033[92m', # green
              '\033[1m', # bold
              'HOORAY!!!! The only found match is\n\n',
              partial_ratio_above75,
              '\033[0m', # end of colored bold text
              2*'\n',
              '...but please check (manually) for validity ;)'
             )
        bestmatch_partialratio = partial_ratio_above75
    else:
        bestmatch_partialratio = 0
        print("We now have", len(partial_ratio_above75), "matches on the shortlist:", 2*"\n",
              partial_ratio_above75)

We now have 2 matches on the shortlist: 

 [('Hopfenmarkt', 91), ('Marktweg', 77)]


#### Give recommendation 2: don't use a label if it has multiple identical best matches (e.g. 3x 85%, 5x 72%, ...)

If the highest (=first) matching value does not appear twice (=second value), we can immediately define it as our best match.

This condition should also eliminate labels containing frequently occurring words
such as *Str*, *St*, *Markt*, *Am*, *Alter*, *Neuer*, *weg*, *Brücke* etc.
because those labels would be too uncertain for the **initial** rough georeferencing

In [15]:
# check if the first matching value in *partial_ratio_above75* is identical to the second matching value
# --> if no: that's the string with the highest matching value and we can simply define that as the only (best) match :)
# --> if yes: go on to the next code block

if len(partial_ratio_above75) > 1: # if there is more than one match in partial_ratio_above75...
    if partial_ratio_above75[0][1] != partial_ratio_above75[1][1]: # ...and if the first match differs from the second...
        print('\033[92m', # green
              '\033[1m', # bold
              'HOORAY!!!! The only found match is\n\n',
              partial_ratio_above75[0], # ...our match is the first one in the list :)
              '\033[0m', # end of colored bold text
              2*'\n',
              '...but please check (manually) for validity ;)'
             )
        bestmatch_partialratio = partial_ratio_above75[0]
    else:
        print("Please go on with this label and have a look at other conditions")
elif len(partial_ratio_above75) == 0:
    print('\033[91m', # red
          '\033[1m', # bold
          "As i said:", 2*"\n",
          " SKIP THIS LABEL!")
elif len(partial_ratio_above75) == 1:
    print("The best match for partial ratio was already defined in the previous step:", 2*"\n",
          '\033[92m', # green
          '\033[1m', # bold
          bestmatch_partialratio)

  HOORAY!!!! The only found match is

 ('Hopfenmarkt', 91)  

 ...but please check (manually) for validity ;)


For validation purposes: compute the numerical difference between the first and the second best match:
- If the difference is larger than 4%, we can declare the first best match as the only best match
- If the difference is still 0%, we should not use this recognized label as a reference point for a rough georeferencing as the similarity measure is too uncertain
- If the difference is smaller than 4%, we should not use this recognized label as a reference point for a rough georeferencing as the similarity measure is too uncertain

In [16]:
if len(partial_ratio_above75) > 1:
    if partial_ratio_above75[0][1]-partial_ratio_above75[1][1] > 4: # than it's allright :)
        print("The difference between the first and second match is \n",
              '\033[1m', # bold
              abs(partial_ratio_above75[0][1]-partial_ratio_above75[1][1]), "% \n",
              '\033[0m', # end of bold text
              "and is therefore >4. Fine! :) You can use ", 
              '\033[1m', # bold
              '\033[92m', # green
              bestmatch_partialratio, 
              '\033[0m', # end of bold text
              " as the one and only best match.",
              sep = "")
    elif partial_ratio_above75[0][1]-partial_ratio_above75[1][1] == 0:
        print('\033[1m', # bold
              "Oh no. The difference between the first and second match is 0 %. This is too uncertain!", 2*"\n",
              '\033[91m', # red
              " SKIP THIS LABEL!")
    else:
        print("The difference between the first and second match is \n",
              '\033[1m', # bold
              abs(partial_ratio_above75[0][1]-partial_ratio_above75[1][1]), "% \n",
              '\033[0m', # end of bold text
              "and therefore <4. Hmmmmmmmmmmmmmm :/ Go on with the next step!",
              sep = "")
elif len(partial_ratio_above75) == 0:
    print('\033[91m', # red
          '\033[1m', # bold
          "As i said: ", 2*"\n",
          " SKIP THIS LABEL!")           
elif len(partial_ratio_above75) == 1:
    print("The best match for partial ratio was already defined in the previous step:", 2*"\n",
          '\033[92m', # green
          '\033[1m', # bold
          bestmatch_partialratio)

The difference between the first and second match is 
14% 
and is therefore >4. Fine! :) You can use ('Hopfenmarkt', 91) as the one and only best match.


Give a last recommendation for (not) using this label as a reference point for the subsequent approximate georeferencing.

In [17]:
try:
    bestmatch_partialratio
    
    if bestmatch_partialratio == 0:
        print('\033[91m', # red
              '\033[1m', # bold
              "SKIP THIS LABEL!") 

    elif len(bestmatch_partialratio) > 2:
        partial_ratio_above75_df = pd.DataFrame(partial_ratio_above75,columns=['roadname','percentage'])
        #print(partial_ratio_above75_df)

        # check if duplicate values exist within *partial_ratio_above75_df['percentage']*
        # --> False = no; True = yes
        checkforduplicates = partial_ratio_above75_df.duplicated(subset=['percentage'], keep=False)
        #print(checkforduplicates)

        # if ANY percentage value is duplicate...
        # --> False = no; True = yes
        result = any(element == True for element in checkforduplicates)
        #print(result)

        # ...don't use this label for the initial georeferencing...
        if result == True:
            print('\033[91m', # red
                  '\033[1m', # bold
                  "Don't use this label for the initial georeferencing, it will be too uncertain due to multiple identical matching values!\n\n SKIP IT!")
        # ... and also not if there is no match >= 75%...
        elif partial_ratio_above75_df.empty:
            print("As I said: Don't use this label for the initial georeferencing, it will be too uncertain due to missing matches >= 75%!\n\nSKIP IT!")
        # ...if there is only one match left: that's our match!
        elif len(partial_ratio_above75_df)==1:
            print('\033[92m', # green
                  '\033[1m', # bold
                  'HOORAY!!!! The only found match is\n\n',
                  partial_ratio_above75_df.values.tolist(),
                  '\033[0m', # end of colored bold text
                  2*'\n',
                  '...but please check (manually) for validity ;)'
                 )
            bestmatch_partialratio = partial_ratio_above75_df
        # ...otherwise you can use it for further testing :)
        else:
            print("You can go on with this label and check for other conditions:", 2*"\n", partial_ratio_above75)

    else:
        print("The best match for partial ratio was already defined in the previous step:", 2*"\n",
              '\033[1m', # bold
              '\033[92m', # green
              bestmatch_partialratio)
except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "As i said: \n\n  SKIP THIS LABEL!")

The best match for partial ratio was already defined in the previous step: 

   ('Hopfenmarkt', 91)


### 2) ratio
*fuzz.ratio* computes the Levenshtein Distance between 2 strings (= number of inserted, removed, and altered characters from string 1 to string 2)

In [18]:
ratio = process.extract(ocr_output, strassen_list, scorer=fuzz.ratio, limit=30)
print(ratio)

[('Hopfenmarkt', 83), ('Alsterdorfer Markt', 65), ('Poppenbüttler Markt', 62), ('Lohbrügger Markt', 62), ('Saseler Markt', 62), ('Hopfensack', 61), ('Bergedorfer Markt', 60), ('Klotzenmoor', 58), ('Langenhorner Markt', 58), ('Barmbeker Markt', 57), ('Sander Markt', 56), ('Ottenser Marktplatz', 56), ('Kleine Horst', 56), ('Roosens Park', 56), ('Kleine Marienstraße', 56), ('Großneumarkt', 56), ('Flottbeker Marktweg', 56), ('Lohsepark', 55), ('Zum Markt', 55), ('Kalenbarg', 55), ('Florapark', 55), ('Alter Fischmarkt', 55), ('Stellinger Markt', 55), ('Klotzenmoorstieg', 55), ('Steinbeker Markt', 55), ('Klostergarten', 54), ('Zeppelin-Park', 54), ('Lohmühlenpark', 54), ('Neugrabener Markt', 53), ('Neuer Pferdemarkt', 53)]


#### Give recommendation 1: only use matches with >= 60% out of *ratio*

We have to lower the threshold [%] here a bit because for *ratio* this value has another impact compared to *partial_ratio* 
<br> 
--> 60%
<br><br>
Again, this threshold was manually defined and based on empirical values from tests with different labels.

In [19]:
for r in ratio:
    #print(r[1])
    if all(r[1] < 60 for pr in ratio):
        print('\033[91m', # red
              '\033[1m', # bold
              "Don't use this label for the initial georeferencing, there is no match >= 60%!",
              2*"\n",
              "SKIP IT!")
        break
    elif any(r[1] >= 60 for r in ratio):
        print("There is at least one match with >= 60% for this label :)",
              2*"\n",
              "You can go on with this label and check for other conditions"
             )
        break

There is at least one match with >= 60% for this label :) 

 You can go on with this label and check for other conditions


Next, print all matches with a similarity >= 60%

In [20]:
ratio_above60 = []

for r in ratio:
    #print(r[1])
    if (r[1] >= 60):
        ratio_above60.append(r)
    elif all(r[1] < 60 for r in ratio):
        print("As i said: There is no match >= 60%! :(")
        break

if ratio_above60: # if the list *ratio_above60* is not empty...
    if len(ratio_above60) == 1:
        print('\033[92m', # green
              '\033[1m', # bold
              'HOORAY!!!! The only found match is\n\n',
              ratio_above60,
              '\033[0m', # end of colored bold text
              2*'\n',
              '...but please check (manually) for validity ;)'
             )
        bestmatch_ratio = ratio_above60
    else:
        bestmatch_ratio = 0
        print("We now have", len(ratio_above60), "matches on the shortlist:", 2*"\n",
              ratio_above60)

We now have 7 matches on the shortlist: 

 [('Hopfenmarkt', 83), ('Alsterdorfer Markt', 65), ('Poppenbüttler Markt', 62), ('Lohbrügger Markt', 62), ('Saseler Markt', 62), ('Hopfensack', 61), ('Bergedorfer Markt', 60)]


#### Give recommendation 2: don't use a label if it has multiple identical best matches (e.g. 3x 85%, 5x 72%, ...)

This condition should also eliminate labels containing frequently occurring words
such as *Str*, *St*, *Markt*, *Am*, *Alter*, *Neuer*, *weg*, *Brücke* etc.
because those labels would be too uncertain for the **initial** rough georeferencing

In [21]:
# if the difference between the first and the second match is <= 4:
# we can put both on the shortlist

if len(ratio_above60) > 1:
    if abs(ratio_above60[0][1] - ratio_above60[1][1]) <= 4:
        bestmatch_ratio = list([ratio_above60[0], ratio_above60[1]]) # overwrite bestmatch_ratio
        print("we now have multiple matches on the shortlist:\n\n",
              '\033[1m', # bold
              bestmatch_ratio[0], bestmatch_ratio[1])
    else:
        bestmatch_ratio = list([ratio_above60[0]])
        print('\033[92m', # green
              '\033[1m', # bold
              'HOORAY!!!! The only found match is\n\n',
              bestmatch_ratio,
              '\033[0m', # end of colored bold text
              2*'\n',
              '...but please check (manually) for validity ;)'
         )
elif len(ratio_above60) == 1:
    print("The best match for ratio was already defined in the previous step:", 2*"\n",
          '\033[1m', # bold
          '\033[92m', # green
          bestmatch_ratio)

  HOORAY!!!! The only found match is

 [('Hopfenmarkt', 83)]  

 ...but please check (manually) for validity ;)


If the highest (=first) matching value does not appear twice (=second value), we can immediately define it as our best match.

In [22]:
# check if the first matching value in *ratio_above60* is identical to the second matching value
# --> if no: that's the string with the highest matching value and we can simply define that as the only match :)
# --> if yes: go on to the next coding block

try:
    bestmatch_ratio 

    if len(bestmatch_ratio) > 1:
        if bestmatch_ratio[0][1] != bestmatch_ratio[1][1]:  
            print('\033[92m', # green
                    '\033[1m', # bold
                    'HOORAY!!!! The only found match is\n\n',
                    bestmatch_ratio[0],
                    '\033[0m', # end of colored bold text
                    2*'\n',
                    '...but please check (manually) for validity ;)'
                )
            bestmatch_ratio = bestmatch_ratio[0]
    else:
        print("The best match for ratio was already defined in the previous step:", 2*"\n",
              '\033[1m', # bold
              '\033[92m', # green
              bestmatch_ratio)

except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "Don't use this label for the initial georeferencing, as there is no best match for ratio!\n\n  SKIP IT!")

The best match for ratio was already defined in the previous step: 

   [('Hopfenmarkt', 83)]


Give a last recommendation for (not) using this label as a reference point for the subsequent approximate georeferencing.

In [23]:
try:
    bestmatch_ratio 

    if len(bestmatch_ratio) > 1:
        if ratio_above60[0][1] == ratio_above60[1][1]:
            ratio_above60_df = pd.DataFrame(ratio_above60,columns=['roadname','percentage'])
            #print(ratio_above60_df)

            # check if duplicate values exist within *ratio_above60_df['percentage']*
            # --> False = no; True = yes
            checkforduplicates_r = ratio_above60_df.duplicated(subset=['percentage'], keep=False)
            #print(checkforduplicates_r)

            # if ANY percentage value is duplicate...
            # --> False = no; True = yes
            result_r = any(element == True for element in checkforduplicates_r)
            #print(result_r)

            # ...don't use this label for the initial georeferencing...
            if result_r == True:
                print('\033[91m', # red
                      '\033[1m', # bold
                      "Don't use this label for the initial georeferencing, it will be too uncertain due to", "\n",
                      " multiple identical matching values in ratio!\n\n SKIP IT!")
            # ... and also not if there is no match >= 60%...
            elif ratio_above60_df.empty:
                print("As I said: Don't use this label for the initial georeferencing, it will be too uncertain due to missing matches >= 60%!\n\nSKIP IT!")
            # ...if there is only one match left: that's our match!
            elif len(ratio_above60_df)==1:
                print('\033[92m', # green
                      '\033[1m', # bold
                      'HOORAY!!!! The only found match is\n\n',
                      ratio_above60_df.values.tolist(),
                      '\033[0m', # end of colored bold text
                      2*'\n',
                      '...but please check (manually) for validity ;)'
                     )
                bestmatch_ratio = ratio_above60_df
            # ...otherwise you can use it for further testing :)
            else:
                print("You can go on with this label and check for other conditions:", 2*" \n", ratio_above60)
        else:
            print("The best match for ratio was already defined in the previous step:", 2*"\n",
            '\033[1m', # bold
            '\033[92m', # green
            bestmatch_ratio)
    else:
        print("The best match for ratio was already defined in the previous step:", 2*"\n",
            '\033[1m', # bold
            '\033[92m', # green
            bestmatch_ratio)
        
except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "Don't use this label for the initial georeferencing, as there is no best match for ratio!\n\n  SKIP IT!")

The best match for ratio was already defined in the previous step: 

   [('Hopfenmarkt', 83)]


## compare between the results of *ratio* (*bestmatch_ratio*) and *partial_ratio* (*bestmatch_partialratio*)
if there does not exist a distinct match until now. By doing that, we can define the best match for a recognized label from the historical map: If the best match from ratio and partial_ratio is identical, we completely trust the algorithm and define this best match as the corresponding street or place name.

In [24]:
#bestmatch_partialratio

In [25]:
#bestmatch_ratio

In [26]:
try:
    bestmatch_partialratio
    if type(bestmatch_partialratio) == list: # if bestmatch_partialratio is a list...
        #print('is a list')
        bestmatch_partialratio = tuple(list(itertools.chain(*bestmatch_partialratio))) # ...make a single tuple out of it.
        print(bestmatch_partialratio)
    else:
        print(bestmatch_partialratio)
except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "I told you! You should not use this label for an initial georeferencing as there is no match for partial ratio!")

('Hopfenmarkt', 91)


In [27]:
try:
    if type(bestmatch_ratio) == list: # if bestmatch_ratio is a list...
        bestmatch_ratio = tuple(list(itertools.chain(*bestmatch_ratio))) # ...make a single tuple out of it.
        print(bestmatch_ratio)
    else:
        print(bestmatch_ratio)
except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "I told you! You should not use this label for an initial georeferencing as there is no match for ratio!")

('Hopfenmarkt', 83)


In [28]:
try:
    bestmatch_partialratio
    bestmatch_ratio
    
    if bestmatch_partialratio != 0:
        if bestmatch_ratio[0] == bestmatch_partialratio[0]:
            print ('\033[1m', # bold
                   "YIPPI!!! :) The best matching strings from ratio and partial_ratio are identical:", 2*"\n",
                   '\033[92m', # green
                   bestmatch_partialratio[0], "-", bestmatch_ratio[0], 2*"\n",
                   '\033[0m', # end of colored bold text
                   "bestmatch from partial ratio:", bestmatch_partialratio[0], bestmatch_partialratio[1], "%", "\n",
                   " bestmatch from ratio:", bestmatch_ratio[0], bestmatch_ratio[1], "%", 2*"\n",
                   " and in total (sum):", bestmatch_ratio[1]+bestmatch_partialratio[1], "%", "\n"
                   "  on average:", (bestmatch_ratio[1]+bestmatch_partialratio[1])/2, "%", 2*"\n"
                   '\033[1m', # bold
                   "Use this label for an intial georeferencing!"
                  )
        else:
            print('\033[1m', # bold
                  '\033[91m', # red
                  "Go on with another label and don't use this one for the initial georeferencing \n",
                  '\033[0m', # end of colored bold text
                  "because the best matching strings from ratio and partial_ratio are not identical - therefore, it's too uncertain:",
                  2*"\n",
                  bestmatch_partialratio[0], "vs.",
                  bestmatch_ratio[0]
                 )
    elif bestmatch_partialratio == 0:
        print('\033[1m', # bold
              "There is no best match for partial ratio as it has too many similar matches.", 2*"\n",
              '\033[91m', # red
              "Don't use this label for an initial georeferencing!")
        
except NameError:
    print('\033[91m', # red
          '\033[1m', # bold
          "Don't use this label for an initial georeferencing!")

 YIPPI!!! :) The best matching strings from ratio and partial_ratio are identical: 

  Hopfenmarkt - Hopfenmarkt 

  bestmatch from partial ratio: Hopfenmarkt 91 % 
  bestmatch from ratio: Hopfenmarkt 83 % 

  and in total (sum): 174 % 
  on average: 87.0 % 

 Use this label for an intial georeferencing!


## subsequent approximate georeferencing

As a following step, an approximate georeferencing (as described in the paper's section 3.6) can be performed with QGIS Georeferencer. With the help of the best overall matches (throughout the whole map), the centroids of the bounding boxes extracted in the text detection step with Strabo can be referenced to the "centroid" of the respective street or place (column 'geometry_centroid' in *strassennetz*) whose original dataset already has a coordinate system. As a result, an approximate and initial georeferencing of the historical map may be generated:
![georeferenced](fig06.png)